In [2]:
%pwd

'c:\\Users\\tizia\\Documents\\Desktop\\MLOps\\TextSummariser'

In [1]:
import os
os.chdir("../")
%pwd

'c:\\Users\\tizia\\Documents\\Desktop\\MLOps\\TextSummariser'

In [15]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

In [4]:
from src.TextSummarizer.constants import *
from src.TextSummarizer.utils.common import read_yaml, create_directories

In [14]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        
        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path=config.model_path,
            tokenizer_path=config.tokenizer_path,
            metric_file_name=config.metric_file_name
        )
        return model_evaluation_config

In [21]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
import pandas as pd
from tqdm import tqdm
from datasets import load_from_disk


In [ ]:
import evaluate

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def generate_batch_sized_chunks(self, list_of_elements, batch_size):
        """Yield successive n-sized chunks from list_of_elements."""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i:i + batch_size]

    def calculate_metric_on_test_ds(self, dataset, metric, model, tokenizer,
                                    batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu",
                                    column_text="article",
                                    column_summary="highlights"):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(target_batches)):
            inputs = tokenizer(article_batch, max_length=1024, return_tensors="pt", padding="max_length", truncation=True)

            summaries = model.generate(input_ids = inputs["input_ids"].to(device),
                                       attention_mask=inputs["attention_mask"].to(device),
                                       length_penalty=0.8,num_beams=8, max_length=128)
            decoded_summaries = [tokenizer.decode(s,skip_special_connections=True,
                                                  clean_up_tokenization_space=True)
                                                  for s in summaries]
            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

            metric.add_batch(predictions=decoded_summaries, references=target_batch)

            score = metric.compute()

            return score
        
    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)

        # loading dataset
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

        rouge_metric = evaluate.load('rouge')

        score = self.calculate_metric_on_test_ds(
            dataset=dataset_samsum_pt["test"][0:10],
            metric=rouge_metric, model=model_pegasus, tokenizer=tokenizer, batch_size =2,
            column_text = 'dialogue', column_summary= 'summary')
        
        # Directly use the scores without accessing femature or imid
        rouge_dict = {rn: score[rn] for rn in rouge_names}

        df = pd.DataFrame(rouge_dict, index=['pegasus'])
        df.to_csv(self.config.metric_file_name, index=False)


In [27]:
config = ConfigurationManager()
model_evaluation_config = config.get_model_evaluation_config()
model_evaluation = ModelEvaluation(config=model_evaluation_config)
model_evaluation.evaluate()

[2026-01-13 15:07:41,376: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-01-13 15:07:41,379: INFO: common: yaml file: params.yaml loaded successfully]
[2026-01-13 15:07:41,379: INFO: common: created directory at: artifacts]
[2026-01-13 15:07:41,386: INFO: common: created directory at: artifacts/model_evaluation]


  0%|          | 0/5 [00:28<?, ?it/s]


ValueError: Mismatch in the number of predictions (2) and references (5)

In [19]:
%pwd

'c:\\Users\\tizia\\Documents\\Desktop\\MLOps\\TextSummariser'